In [1]:
# the necessary imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
import sklearn.tree as scikit
import math
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
import statsmodels.api as sm

In [2]:
import seaborn as sn

In [3]:
data=pd.read_csv('C:/Users/SnarfBob/Desktop/CSci 5523/group proj/train.csv')

In [3]:
data_no_re=pd.read_csv('C:/Users/SnarfBob/Desktop/CSci 5523/group proj/train no redundant.csv')

In [4]:
data_no_re_low=pd.read_csv('C:/Users/SnarfBob/Desktop/CSci 5523/group proj/train no redundant or low.csv')

In [5]:
print(data_no_re_low.shape)
print(data_no_re_low.head())

(61878, 65)
   id  f1  f8  f9  f11  f13  f14  f15  f16  f17  ...  f79  f83  f85  f86  f87  \
0   1   1   0   0    1    0    0    0    0    2  ...    2    0    1    0    0   
1   2   0   1   0    0    0    0    0    0    0  ...    0    0    0    0    0   
2   3   0   1   0    0    0    0    0    0    1  ...    0    0    0    0    0   
3   4   1   0   0    1    1    0    0    1    1  ...    1    0    0    1    2   
4   5   0   0   0    0    0    0    0    0    4  ...    4    0    1    0    0   

   f88  f90  f91  f92  class  
0    0    0    0    0      1  
1    0    0    0    0      1  
2    0    0    0    0      1  
3    0    0    0    0      1  
4    0    1    0    0      1  

[5 rows x 65 columns]


In [4]:
datax=data.iloc[0:61878,1:94]
datay=data.iloc[0:61878,94]
print(datax.head())
print(datay.head())

   f1  f2  f3  f4  f5  f6  f7  f8  f9  f10  ...  f84  f85  f86  f87  f88  f89  \
0   1   0   0   0   0   0   0   0   0    0  ...    0    1    0    0    0    0   
1   0   0   0   0   0   0   0   1   0    0  ...    0    0    0    0    0    0   
2   0   0   0   0   0   0   0   1   0    0  ...    0    0    0    0    0    0   
3   1   0   0   1   6   1   5   0   0    1  ...   22    0    1    2    0    0   
4   0   0   0   0   0   0   0   0   0    0  ...    0    1    0    0    0    0   

   f90  f91  f92  f93  
0    0    0    0    0  
1    0    0    0    0  
2    0    0    0    0  
3    0    0    0    0  
4    1    0    0    0  

[5 rows x 93 columns]
0    1
1    1
2    1
3    1
4    1
Name: class, dtype: int64


In [7]:
datax_nore=data_no_re.iloc[0:61878,1:87]
datay_nore=data_no_re.iloc[0:61878,87]
print(datax_nore.head())
print(datay_nore.head())

   f1  f2  f5  f6  f7  f8  f9  f10  f11  f12  ...  f84  f85  f86  f87  f88  \
0   1   0   0   0   0   0   0    0    1    0  ...    0    1    0    0    0   
1   0   0   0   0   0   1   0    0    0    0  ...    0    0    0    0    0   
2   0   0   0   0   0   1   0    0    0    0  ...    0    0    0    0    0   
3   1   0   6   1   5   0   0    1    1    0  ...   22    0    1    2    0   
4   0   0   0   0   0   0   0    0    0    0  ...    0    1    0    0    0   

   f89  f90  f91  f92  f93  
0    0    0    0    0    0  
1    0    0    0    0    0  
2    0    0    0    0    0  
3    0    0    0    0    0  
4    0    1    0    0    0  

[5 rows x 86 columns]
0    1
1    1
2    1
3    1
4    1
Name: class, dtype: int64


In [8]:
datax_norelow=data_no_re_low.iloc[0:61878,1:64]
datay_norelow=data_no_re_low.iloc[0:61878,64]
print(datax_norelow.head())
print(datay_norelow.head())

   f1  f8  f9  f11  f13  f14  f15  f16  f17  f18  ...  f78  f79  f83  f85  \
0   1   0   0    1    0    0    0    0    2    0  ...    0    2    0    1   
1   0   1   0    0    0    0    0    0    0    2  ...    1    0    0    0   
2   0   1   0    0    0    0    0    0    1    0  ...    0    0    0    0   
3   1   0   0    1    1    0    0    1    1    0  ...    0    1    0    0   
4   0   0   0    0    0    0    0    0    4    0  ...    0    4    0    1   

   f86  f87  f88  f90  f91  f92  
0    0    0    0    0    0    0  
1    0    0    0    0    0    0  
2    0    0    0    0    0    0  
3    1    2    0    0    0    0  
4    0    0    0    1    0    0  

[5 rows x 63 columns]
0    1
1    1
2    1
3    1
4    1
Name: class, dtype: int64


In [110]:
# create correlation matrix
cormat = datax.corr()

           f1        f2        f3        f4        f5        f6        f7  \
f1   1.000000  0.031332 -0.027807 -0.027529  0.042973  0.043603  0.298952   
f2   0.031332  1.000000  0.082573  0.134987  0.020926  0.041343  0.222386   
f3  -0.027807  0.082573  1.000000  0.583523  0.010880  0.004288  0.001294   
f4  -0.027529  0.134987  0.583523  1.000000  0.017290  0.014059  0.014490   
f5   0.042973  0.020926  0.010880  0.017290  1.000000  0.145355  0.075047   
..        ...       ...       ...       ...       ...       ...       ...   
f89  0.096851  0.105527  0.174781  0.183715  0.119951  0.035042  0.063511   
f90  0.010310  0.515022 -0.015068  0.009454  0.004842  0.054034  0.129578   
f91  0.037264  0.026383 -0.012417 -0.010312  0.012012  0.012465  0.068506   
f92  0.054777 -0.008219  0.066921  0.087631  0.065331  0.015479 -0.032261   
f93  0.081783  0.054593  0.006814  0.015746  0.002038  0.008521  0.034912   

           f8        f9       f10  ...       f84       f85       f86  \
f1 

In [120]:
cormat1=pd.DataFrame(cormat)

In [121]:
cormat1.to_csv("cormat1.csv")

In [9]:
oversampler = RandomOverSampler(sampling_strategy='not majority', random_state=0)

In [60]:
undersampler = RandomUnderSampler(sampling_strategy='not minority', random_state=0)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(datax, datay, test_size=0.25, random_state=0, stratify=datay)

In [11]:
x_train_nore, x_test_nore, y_train_nore, y_test_nore = train_test_split(datax_nore, datay_nore, test_size=0.25, random_state=0, stratify=datay_nore)

In [12]:
x_train_norelow, x_test_norelow, y_train_norelow, y_test_norelow = train_test_split(datax_norelow, datay_norelow, test_size=0.25, random_state=0, stratify=datay_norelow)

In [13]:
overx, overy = oversampler.fit_resample(x_train, y_train)

In [14]:
overx_nore, overy_nore = oversampler.fit_resample(x_train_nore, y_train_nore)

In [15]:
overx_norelow, overy_norelow = oversampler.fit_resample(x_train_norelow, y_train_norelow)

In [63]:
underx, undery = undersampler.fit_resample(x_train, y_train)

In [26]:
overtree = DecisionTreeClassifier(criterion='entropy', splitter='best')
overtree_fit=overtree.fit(overx, overy)
y_pred_overtree = overtree_fit.predict(x_test)
accuracy_score(y_test, y_pred_overtree)

In [38]:
#from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(overx, overy)
y_pred_knn = neigh.predict(x_test)
accuracy_score(y_test_nore, y_pred_knn)

0.741305753070459

In [39]:
neigh_nore = KNeighborsClassifier(n_neighbors=5)
neigh_nore.fit(overx_nore, overy_nore)
y_pred_knn_nore = neigh_nore.predict(x_test_nore)
accuracy_score(y_test_nore, y_pred_knn_nore)

0.7398190045248869

In [46]:
neigh_norelow = KNeighborsClassifier(n_neighbors=5)
neigh_norelow.fit(overx_norelow, overy_norelow)
y_pred_knn_norelow = neigh_norelow.predict(x_test_norelow)
accuracy_score(y_test_norelow, y_pred_knn_norelow)

0.7381383322559794

In [97]:
undertree = DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=None, min_samples_split=4, 
                             min_samples_leaf=2, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, 
                             max_leaf_nodes=None, min_impurity_decrease=0.0001, class_weight=None, ccp_alpha=0.0)

In [98]:
undertree_fit=undertree.fit(underx, undery)

In [99]:
y_pred_undertree = undertree_fit.predict(x_test)

In [100]:
accuracy_score(y_test, y_pred_undertree)

0.616677440206852

In [84]:
#since the tree with oversampling did so much better than the tree with undersampling it makes sense to use the tree with
#undersampling to get the importance features
for name, importance in zip(datax.columns, overtree_fit.feature_importances_):
    print(name, importance)

f1 0.005952879488093526
f2 0.0009593706669244358
f3 0.0011562911265328697
f4 0.0016007879529752716
f5 0.002653886790135046
f6 7.329762993138692e-05
f7 0.0017978630626584958
f8 0.013325419287532395
f9 0.018524016823369783
f10 0.0027095675465508024
f11 0.05295369289698497
f12 0.0009579397797491823
f13 0.004258072348896897
f14 0.05063072795006323
f15 0.04659885093026308
f16 0.00918496282869007
f17 0.012954930930400226
f18 0.0048171970301264895
f19 0.0012439101354115233
f20 0.004120644682556249
f21 0.0018058966854826231
f22 0.003485429098503257
f23 0.001845931291529218
f24 0.015144174518835603
f25 0.022873645436304275
f26 0.015328468197954408
f27 0.0022672883022551737
f28 0.0007847511298542095
f29 0.0034326284771246073
f30 0.009435639043379627
f31 0.0005371430945347266
f32 0.013270920444745723
f33 0.007445847347126364
f34 0.13601641645188764
f35 0.003503872242146635
f36 0.027740425462313693
f37 0.00480333720996706
f38 0.005338589809540175
f39 0.03723098062134488
f40 0.02100616617229967
f41

In [16]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 100,
                           n_jobs = -1,
                           max_depth=None,
                           min_samples_split=2,
                           min_samples_leaf=1,
                           bootstrap = True,
                           random_state = 0)

In [127]:
rf.fit(overx, overy)

RandomForestClassifier(n_jobs=-1, random_state=0)

In [128]:
y_pred_rf = rf.predict(x_test)
accuracy_score(y_test, y_pred_rf)

0.8163542340012928

In [130]:
for name, importance in zip(datax.columns, rf.feature_importances_):
    print(name, importance)

f1 0.007707209217637041
f2 0.003248592509896754
f3 0.008933191529731344
f4 0.006629392809578563
f5 0.0024951928018567392
f6 0.0008908347619172575
f7 0.003681972359597744
f8 0.015312908297742442
f9 0.014012878844305444
f10 0.0046303208413537335
f11 0.026734896901676863
f12 0.0026735663466007716
f13 0.006814541252453745
f14 0.026388115366510364
f15 0.021735298344925838
f16 0.011906331209617442
f17 0.015245666359782577
f18 0.006482788189036314
f19 0.0040669637686386
f20 0.006492524889075433
f21 0.0035600930946410253
f22 0.006471730177274356
f23 0.004780158154707544
f24 0.02047539226218694
f25 0.0302187387966355
f26 0.016202251540012435
f27 0.007388537870285722
f28 0.002019979346448447
f29 0.00505157120999647
f30 0.007602626328326304
f31 0.0015886427426779807
f32 0.014543241711705348
f33 0.01090103841901524
f34 0.0858320465259908
f35 0.0073451856466042265
f36 0.020712235655421997
f37 0.006643112197430447
f38 0.007771207928028711
f39 0.021217407941523804
f40 0.023092684006603147
f41 0.00813

In [17]:
rf.fit(overx_nore, overy_nore)

RandomForestClassifier(n_jobs=-1, random_state=0)

In [18]:
y_pred_rf_nore = rf.predict(x_test_nore)
accuracy_score(y_test_nore, y_pred_rf_nore)

0.8115061409179056

In [19]:
rf.fit(overx_norelow, overy_norelow)
y_pred_rf_norelow = rf.predict(x_test_norelow)
accuracy_score(y_test_norelow, y_pred_rf_norelow)

0.8145442792501616

In [22]:
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB()
GNB.fit(overx, overy)
y_pred_GNB_over=GNB.predict(x_test)
accuracy_score(y_test, y_pred_GNB_over)

0.6023917259211377

In [24]:
GNB.fit(overx_nore, overy_nore)
y_pred_GNB_nore=GNB.predict(x_test_nore)
accuracy_score(y_test, y_pred_GNB_nore)

0.6046541693600517

In [25]:
GNB.fit(overx_norelow, overy_norelow)
y_pred_GNB_norelow=GNB.predict(x_test_norelow)
accuracy_score(y_test, y_pred_GNB_norelow)

0.629541047188106

In [ ]:
small=np.zeros(shape=(93,93), dtype=float)
for i in range(93):
    for j in range(93):
        xixj=datax.iloc[0:61878,i]*datax.iloc[0:61878,j]
        x = sm.add_constant(xixj)
        model = sm.OLS(datay,x)
        results = model.fit()
        small[i,j]=results.tvalues[0]

In [ ]:
pd.DataFrame(small).to_csv("small.csv")

In [5]:
small=np.zeros(shape=(93,1), dtype=float)
for i in range(93):
    xi=datax.iloc[0:61878,i]
    x = sm.add_constant(xi)
    model = sm.OLS(datay,x)
    results = model.fit()
    small[i]=results.tvalues[0]

In [6]:
pd.DataFrame(small).to_csv("small.csv")

In [12]:
#import rulekit - commenting out everything related to RIPPER because it won't run if the user doesn't have JAVE JRE installed
#from rulekit import RuleKit
#RuleKit.init()
#!pip install rulekit --user 
#!python -m rulekit download_jar

# from rulekit.classification import RuleClassifier
# from rulekit.params import Measures
# from IPython.display import display

In [ ]:
# # Ripper
# clf = RuleClassifier(
#     induction_measure=Measures.CFoil,
#     pruning_measure=Measures.Ripper,
#     voting_measure=Measures.Ripper,
# )
# clf.fit(x_train, y_train)
# c2_ruleset = clf.model
# y_pred = clf.predict(x_test, return_metrics=False)



In [ ]:
# metrics.accuracy_score(y_test, y_pred)

In [ ]:
# metrics.confusion_matrix(y_test, y_pred)